In [ ]:
import pickle
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import load_model,Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket('stacked-models-tpu')
# Then do other things...
blob = bucket.get_blob('sentiment/')

# print(blob.download_as_string())
# blob.upload_from_string('New contents!')
# blob2 = bucket.blob('remote/path/storage.txt')
# blob2.upload_from_filename(filename='/local/path.txt')


In [ ]:
def load_all_models(names):
  with tf.device('/job:localhost'):
      # for name in names:
      # # define filename for this ensemble
      #   filename = '/content/drive/MyDrive/sentiment models/'+name
      # # load model from file
      # models = [load_model('/content/drive/MyDrive/sarcasm models/'+name) for name in names]
    models = [load_model('gs://stacked-models-tpu/sentiment/'+name) for name in names]
      # add to list of members
        # all_models.append(model)
        # print('>loaded %s' % name)
  return models


In [ ]:
def define_stacked_model(names,members):
	# update all layers in all models to not be trainable
  for i in range(len(members)):
    model = members[i]
    # model.input._name = names[i].replace('.h5','')+'_'+model.input.name
    for layer in model.layers:
      # make not trainable
      layer.trainable = True
      # rename to avoid 'unique layer name' issue
      layer._name = names[i].replace('.h5','')+'_' + 'ensemble'+'_'+str(i+1) + '_' + layer.name
    # define multi-headed input
  ensemble_visible = [model.input for model in members]
  # concatenate merge output from each model
  ensemble_outputs = [model.output for model in members]
  merge = concatenate(ensemble_outputs)
  hidden = Dense(16, activation='relu')(merge)
  output = Dense(5, activation='softmax')(hidden)
  model = Model(inputs=ensemble_visible, outputs=output)
  # plot graph of ensemble
  # plot_model(model, show_shapes=True, to_file='model_graph.png')
  # compile
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
stacked6 = ['sentiment_CONV1D.h5','sentiment_BiLSTM.h5','sentiment_CONV1D_BiLSTM.h5','sentiment_SEP_CONV1D.h5','sentiment_BiGRU.h5']
with strategy.scope():
      members = load_all_models(stacked6)
      stacked = define_stacked_model(stacked6,members)

In [ ]:
# train_data_with_folds_variant_#6.pickle  val_data_with_folds_variant_#6.pickle
import pickle
with open('train_data_with_folds_variant_#6.pickle','rb') as f:
  train_data = pickle.load(f)
with open('val_data_with_folds_variant_#6.pickle','rb') as f:
  val_data = pickle.load(f)

In [ ]:
for i in range(5):
  stacked.fit(x = train_data[i][0],y = train_data[i][1],validation_data=(val_data[i][0],val_data[i][1]),epochs=100,callbacks=[EarlyStopping(monitor='val_loss'),ModelCheckpoint(filepath='trained_sentiment_#6.h5', monitor='val_loss', save_best_only=True))